# 02_feature_engineering
- 目标：快速查看坡度、道路/电网距离、土地利用分布，辅助后续阈值选择。
- 基准栅格：`data/interim/irradiance_reproj.tif`（4.4km），所有特征已对齐。
- 依赖：环境 `ai_renewable`（geopandas/rasterio/matplotlib）。

小步说明：导入库与绘图风格。
输入：无（仅依赖已安装的库）。
输出：绘图所需的基础工具与样式。

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

plt.style.use("ggplot")  # 使用简洁风格
plt.rcParams["figure.figsize"] = (8, 5)

小步说明：补充项目根目录到 sys.path，确保可导入 src.* 模块。
输入：无（使用当前工作目录推断）。
输出：更新 sys.path。

In [ ]:
from pathlib import Path
import sys

ROOT = Path.cwd().resolve()
if not (ROOT / "src").exists():
    ROOT = ROOT.parent  # 若当前不含 src，尝试上一层
if str(ROOT) not in sys.path:
    sys.path.append(str(ROOT))
print("Project root:", ROOT)

小步说明：加载对齐后的坡度/距离/土地利用栅格。
输入：`data/interim/slope_resampled_to_irradiance.tif`、`dist_roads.tif`、`dist_grid.tif`、`landcover_resampled_to_irradiance.tif`。
输出：掩膜数组（numpy.ma）和 profile，便于后续绘图与校验。

In [ ]:
from src.features import (
    load_dist_grid,
    load_dist_roads,
    load_landcover,
    load_slope,
)

slope, slope_profile = load_slope()
dist_roads, dist_roads_profile = load_dist_roads()
dist_grid, dist_grid_profile = load_dist_grid()
landcover, landcover_profile = load_landcover()

小步说明：封装一个简单直方图函数。
输入：掩膜数组、标题、单位/标签。
输出：单张直方图，用于快速查看分布。

In [ ]:
def plot_hist(masked_arr, title, bins=20, xlabel=None):
    data = np.ma.compressed(masked_arr)  # 去掉掩膜后的有效值
    plt.hist(data, bins=bins, color="steelblue", alpha=0.8)
    plt.title(title)
    plt.xlabel(xlabel or title)
    plt.ylabel("Count")
    plt.show()

小步说明：查看坡度与距离的分布，用于初步阈值判断。
输入：`slope`、`dist_roads`、`dist_grid` 掩膜数组。
输出：三张直方图。

In [ ]:
plot_hist(slope, "Slope (degrees)", bins=20, xlabel="Degrees")
plot_hist(dist_roads, "Road Distance (m)", bins=20, xlabel="Meters")
plot_hist(dist_grid, "Grid Distance (m)", bins=20, xlabel="Meters")

小步说明：统计土地利用类别分布（像元计数），便于筛选候选类别。
输入：`landcover` 掩膜数组。
输出：类别值与对应计数。

In [ ]:
vals, counts = np.unique(np.ma.compressed(landcover), return_counts=True)
print("Landcover class counts:")
for v, c in zip(vals, counts):
    print(f"class {int(v)}: {c}")